# Webscrapping data from http://accent.gmu.edu - Speech Accent Archive

In [163]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [179]:
# Scrapping all the files with Chinese accent
base_url = "http://accent.gmu.edu/"
audio_file_path='../Data/archive-2/recordings/recordings/'

#different URLs to scrape from to get all files
#replace URL and run remaining notebook one at a time to get details and download files
#1)"http://accent.gmu.edu/browse_language.php?function=find&language=chinese"
#2)"http://accent.gmu.edu/browse_language.php?function=find&language=gujarati"
#3)"http://accent.gmu.edu/browse_language.php?function=find&language=hindi"
#4)"http://accent.gmu.edu/browse_language.php?function=find&language=kannada"
#5)"http://accent.gmu.edu/browse_language.php?function=find&language=malayalam"
#6)"http://accent.gmu.edu/browse_language.php?function=find&language=panjabi"
#7)"http://accent.gmu.edu/browse_language.php?function=find&language=tamil"
#8)"http://accent.gmu.edu/browse_language.php?function=find&language=telugu"
#9)"http://accent.gmu.edu/browse_language.php?function=find&language=bengali"
#10)"http://accent.gmu.edu/browse_language.php?function=find&language=mandarin"
#11)"http://accent.gmu.edu/browse_language.php?function=find&language=english"

url ="http://accent.gmu.edu/browse_language.php?function=find&language=english"
res=requests.get(url)

In [180]:
soup = BeautifulSoup(res.content, 'lxml')

In [181]:
paras=soup.find_all('p')
#paras.pop(0)



In [182]:
paras.pop(0)

<p><img alt="" src="//chnmdev.gmu.edu/piwik/piwik.php?idsite=2" style="border:0;"/></p>

In [183]:
links = [{'link':p.find('a').attrs['href'] , 'fileName':p.find('a').text} for p in paras]   
len(links)

646

In [184]:
links

[{'link': 'browse_language.php?function=detail&speakerid=61',
  'fileName': 'english1,'},
 {'link': 'browse_language.php?function=detail&speakerid=77',
  'fileName': 'english2,'},
 {'link': 'browse_language.php?function=detail&speakerid=88',
  'fileName': 'english3,'},
 {'link': 'browse_language.php?function=detail&speakerid=99',
  'fileName': 'english4,'},
 {'link': 'browse_language.php?function=detail&speakerid=110',
  'fileName': 'english5,'},
 {'link': 'browse_language.php?function=detail&speakerid=121',
  'fileName': 'english6,'},
 {'link': 'browse_language.php?function=detail&speakerid=132',
  'fileName': 'english7,'},
 {'link': 'browse_language.php?function=detail&speakerid=143',
  'fileName': 'english8,'},
 {'link': 'browse_language.php?function=detail&speakerid=154',
  'fileName': 'english9,'},
 {'link': 'browse_language.php?function=detail&speakerid=62',
  'fileName': 'english10,'},
 {'link': 'browse_language.php?function=detail&speakerid=68',
  'fileName': 'english11,'},
 {'

In [186]:
# Function to download file from website 
# Source : https://www.pluralsight.com/guides/guide-scraping-media-from-the-web-python

def downloadFile(AFileName):
    # extract file name from AFileName
    filename = AFileName.split("/")[-1] 
    # download audio using GET
    rawAudio = requests.get(AFileName, stream=True)
    # save the audio recieved into the file
    with open(audio_file_path+filename, 'wb') as fd:
        for chunk in rawAudio.iter_content(chunk_size=1024):
            fd.write(chunk)
    return filename

In [187]:
def get_details(link):
    lnk=link['link']
    urld=base_url+lnk
    resd=requests.get(urld)
    soupd=BeautifulSoup(resd.content, 'lxml')
    audiolink=soupd.find('audio').find('source').attrs['src']
    ls=soupd.find_all('li')
    details={}
    details['age']=ls[12].text[10:10+ls[12].text.index(',')-1]
    details['age_onset']=ls[13].text[ls[13].text.index(':')+1:]
    details['birthplace']=ls[9].text[13:]
    
    #downloading audio file
    
    filename = downloadFile(base_url+ audiolink)
    
    details['filename']=filename[0:filename.index('.')]
    #Use this for english only
    details['native_language'] = ls[10].text[ls[10].text.index(':')+2:ls[10].text.index(':')+9]
    #details['native_language'] = ls[10].text[ls[10].text.index(':')+2:]
    details['sex']=ls[12].text[10+ls[12].text.index(',')+1:]
    
    details['speakerid']=lnk[lnk.index('speakerid=')+10:]
    details['country']=ls[15].text[ls[15].text.index(':')+2:]
    #details['country']='china'
    details['file_missing?']='FALSE'
    

    
    time.sleep(1)
    return details

In [190]:
# create list of dict key value
all_speaker_details=[get_details(link) for link in links[0:630]]
#all_speaker_details=[get_details(link) for link in links]

In [191]:
len(all_speaker_details)

630

In [192]:
df_details=pd.DataFrame(all_speaker_details)

In [193]:
df_details

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?
0,42,0,"pittsburgh, pennsylvania, usa (map)",english1,english,male,61,usa,FALSE
1,30,0,"birmingham, uk (map)",english2,english,female,77,uk,FALSE
2,26,0,"brisbane, australia (map)",english3,english,female,88,australia,FALSE
3,53,0,"saint anne's bay, jamaica (map)",english4,english,female,99,"jamaica,usa",FALSE
4,62,0,"fairfax, virginia, usa (map)",english5,english,male,110,usa,FALSE
...,...,...,...,...,...,...,...,...,...
625,21,0,"hammond, indiana, usa (map)",english626,english,female,2806,usa,FALSE
626,18,0,"sydney, australia (map)",english627,english,female,2847,australia,FALSE
627,55,0,"pittsburgh, pennsylvania, usa (map)",english628,english,male,2855,usa,FALSE
628,20,0,"south riding, virginia, usa (map)",english629,english,female,2894,usa,FALSE


In [176]:
#df_details['native_language']='mandarin'

In [194]:
df_details.to_csv('../Data/all_speakers_english1.csv',index=False)